In [1]:
import os

In [2]:
%pwd

'/Users/suman/Desktop/end_to_end_projects/MLOPS_deeplearning/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/suman/Desktop/end_to_end_projects/MLOPS_deeplearning'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from ChickenDisease.constants import *
from ChickenDisease.utils.common import read_yaml , create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        dataingestion_config = self.config.data_ingestion

        create_directories([dataingestion_config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=dataingestion_config.root_dir,
            source_URL=dataingestion_config.source_URL,
            local_data_file=dataingestion_config.local_data_file,
            unzip_dir=dataingestion_config.unzip_dir 
        )

        return data_ingestion_config
      

In [12]:
import os
import urllib.request as request
import zipfile
from ChickenDisease import logger
from ChickenDisease.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! ")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-07 13:25:04,853: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-07 13:25:04,854: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-07 13:25:04,855: INFO: common: created directory at: artifacts]
[2023-09-07 13:25:04,856: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-07 13:26:24,133: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 60D2:1806:6FDC23:8815A0:64F98222
Accept-Ranges: bytes
Date: Thu, 07 Sep 2023 07:56:25 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10249-MAA
X-Cache: MI